In [9]:
#Entrainement du modèle à déployer
#1.Importation des librairies
import pandas as pd 
import numpy as np 

import sklearn
### Train Test Split
from sklearn.model_selection import train_test_split
#Gestion unbalanced
import imblearn
from imblearn.over_sampling import SMOTE
#Classifier
import lightgbm
from lightgbm import LGBMClassifier

### Check Accuracy
from sklearn.metrics import accuracy_score

In [14]:
#Importation des données
path = "/Users/adpro/Desktop/Scoring_Model/data_api.csv"
df = pd.read_csv(path)
df = df.set_index("ID")
df.head()

,EXT_SOURCE_2,DAYS_REGISTRATION,EXT_SOURCE_3,DAYS_ID_PUBLISH,DAYS_BIRTH,DAYS_LAST_PHONE_CHANGE,PAYMENT_RATE,REGION_POPULATION_RELATIVE,Target
ID,,,,,,,,,
100002,0.262949,-3648.0,0.139376,-2120,-9461,-1134.0,0.060749,0.018801,1
100003,0.622246,-1186.0,0.535276,-291,-16765,-828.0,0.027598,0.003541,0
100004,0.555912,-4260.0,0.729567,-2531,-19046,-815.0,0.050000,0.010032,0
100006,0.650442,-9833.0,0.535276,-2437,-19005,-617.0,0.094941,0.008019,0
100007,0.322738,-4311.0,0.535276,-3458,-19932,-1106.0,0.042623,0.028663,0


In [15]:
#pré-proc des données
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

#split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#resampling SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [16]:
#modele
clf = LGBMClassifier(application='binary',
                         is_unbalance=True,
                         metric='auc',
                         max_depth=30,
                         n_estimators=4000,
                         num_leaves=40,
                         subsample=0.1)

In [17]:
#Fiting
clf.fit(X_res,y_res)
#prediction
y_pred=clf.predict(X_test)

In [18]:
score=accuracy_score(y_test,y_pred)
score

0.9055

In [19]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("clf.pkl","wb")
pickle.dump(clf, pickle_out)
pickle_out.close()

In [22]:
import requests
client_test = {
    "EXT_SOURCE_2": 0.6222457752555098,
    "DAYS_REGISTRATION": -1186.0,
    "EXT_SOURCE_3": 0.5352762504724826,
    "DAYS_ID_PUBLISH": -291,
    "DAYS_BIRTH": -16765,
    "DAYS_LAST_PHONE_CHANGE": -828.0,
    "PAYMENT_RATE": 0.0275983231574736,
    "REGION_POPULATION_RELATIVE": 0.0035409999999999
    }
response = requests.post('http://127.0.0.1:8000/predict',json=client_test)
print(response.content)

b'{"prediction":"Client non solvable","probabilite":0.0}'
